# Different long-term thermal comfort indices for continuous monitoring

In [1]:
from temperature_range_percentage import *

In [2]:
temp_range?


Signature: temp_range(l, u, f)
Docstring:
Calculate the percentage of occupied time outside a temeprature range.
The occupied time is supposed to be from 9 am to 5 pm at weekdays. 
The data file type should be CSV.
The first column of the CSV file should be time.
The second column of the CSV file should be temperature.

Parameters
----------
l : float
    lower bound of the tempearture range, with same units of the data
u : float
    upper bound of the temperature range, with same units of the data
f : string
    file path of the CSV dataset

Returns
----------
p : float
    percentage of the time
File:      ~/Documents/GitHub/mortardata_comfort_evaluation/temperature_range_percentage.py
Type:      function


## No.1: Percentage of occupied time outside a temeprature range

In [3]:
Two_East = '/Users/ruijisun/Downloads/20210727 Zone Temperature Data/DBC/Field Bus1.BrowerRadiant2East.ZN2-T.csv'
print("{:.0%}".format(temp_range(72, 81, Two_East)))

82%


In [4]:
Three_West= '/Users/ruijisun/Downloads/20210727 Zone Temperature Data/DBC/Field Bus1.BrowerRadiant3West.ZN1-T.csv'
print("{:.0%}".format(temp_range(72, 81, Three_West)))

38%


In [5]:
Four_East = '/Users/ruijisun/Downloads/20210727 Zone Temperature Data/DBC/Field Bus1.BrowerRadiant4East.ZN2-T.csv'
print("{:.0%}".format(temp_range(72, 81, Four_East)))

40%


## No.2: Mean Temperature 

In [6]:
from temperature_mean import *

In [7]:
temp_mean(Two_East)

67.82025979822039

In [8]:
temp_mean(Three_West)

72.63266124303229

In [9]:
temp_mean(Four_East)

71.57280375565651

## No.3: Temperature Variance 

In [10]:
from temperature_variance import *

In [11]:
temp_var(Two_East)

18.476287426429852

In [12]:
temp_var(Three_West)

25.258746009585497

In [13]:
temp_var(Four_East)

32.218171462457214

## No.4: Percentage of occupied days that daily temperature range are out of the threshold

In [14]:
from temperature_daily_outlier import *

In [16]:
print("{:.0%}".format(temp_daily(2, Two_East)))

49%


In [18]:
print("{:.0%}".format(temp_daily(2, Three_West)))

45%


In [19]:
print("{:.0%}".format(temp_daily(2, Four_East)))

59%
